### Import

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import warnings;warnings.filterwarnings(action='ignore')

### Read data
한 행은 한 건의 공연티켓 예매 정보이다. 콘서트홀의 클래식 공연만을 분석 대상으로 삼는다.

In [2]:
classic = ['교향곡', '클래식', '합창', '독주', '성악', '실내악']
data = pd.read_csv('../data/2023빅콘테스트_어드밴스드리그_예술의전당.csv',
                   parse_dates=['tran_date', 'play_date', 'pre_open_date', 'open_date'])\
       .query('place=="콘서트홀" and genre in @classic').reset_index(drop=True)
data.drop('place', axis=1, inplace=True)
data.shape

(1009551, 23)

### <font color='firebrick'>결측치
- 공연 정보(pre_open_date) 결측치가 있다.<br>
  $\bullet$ pre_open_date의 결측은 선예매 서비스를 제공하지 않았다고 해석한다.

In [3]:
# 결측치 분포
data.isna().sum()

age                   551777
gender                551469
membership_type_1     551469
membership_type_2     639139
membership_type_3     874253
membership_type_4     985052
membership_type_5    1008466
membership_type_6    1009551
tran_date                  0
tran_time                  0
play_date                  0
play_st_time               0
seat                       0
price                      0
ticket_cancel              0
discount_type              0
performance_code           0
pre_open_date         366280
open_date                  0
genre                      0
running_time               0
intermission               0
member_yn                  0
dtype: int64

- 회원 정보(age, gender, membership_type) 결측치가 있다.<br>
  $\bullet$ 회원으로 결측이 없는 데이터(468469)<br>
  $\bullet$ 비회원이며 멤버십을 가입하지 않아 회원 정보가 모두 결측인 데이터(586393)<br>
  $\bullet$ `비회원이나 멤버십을 가입하여 membership_type과 멤버십 가입 시 묻는 gender는 결측이 없는 데이터(314)`
  - 가설1)회원이었으나 탈퇴/만료 회원이 되어 멤버십 정보가 있으나 비회원으로 표기된 데이터이다.
  - 가설2)개인정보 제공동의가 만료된 데이터이다.<br>
    $\bullet$ "개인정보의 보유 및 이용기간은 최종 로그인 기준 1년이며, <br>
    $~~$ 예술의전당 이용고객(유료회원, 입장권 구매, 수강신청)은 최종 이용일 기준 5년입니다."
  - 가설3)단순 데이터 오류다.

In [4]:
member, notmember = data.query('member_yn=="Y"').iloc[:,:8], data.query('member_yn=="N"').iloc[:,:8]
print(f'회원 예매내역: {member.shape[0]}, 비회원 예매내역: {notmember.shape[0]}')

회원 예매내역: 457774, 비회원 예매내역: 551777


In [5]:
# 회원이면 나이, 성별, 멤버십 정보가 있다.
member.isna().sum()

age                       0
gender                    0
membership_type_1         0
membership_type_2     87637
membership_type_3    322522
membership_type_4    433277
membership_type_5    456689
membership_type_6    457774
dtype: int64

In [6]:
# 비회원 중 멤버십 가입이 이루어지지 않은 이의 구매내역이다.
notmember.loc[notmember['gender'].isna()].isna().sum()

age                  551469
gender               551469
membership_type_1    551469
membership_type_2    551469
membership_type_3    551469
membership_type_4    551469
membership_type_5    551469
membership_type_6    551469
dtype: int64

In [7]:
# 비회원이면 나이 정보가 없으며 성별, 멤버십 정보가 있는 예매 건수가 조금 있다.
'''비회원인데 유료회원에 가입되어 있는 것이 440건에 불과함에 상세히 분석하지 않고 제거하도록 한다.'''
print('비회원인데 멤버십 정보가 있는 데이터수: ', notmember.loc[notmember['gender'].notna()].shape[0])
notmember.loc[notmember['gender'].notna()].iloc[:,2:8].drop_duplicates()

# 성별과 멤버십 데이터수가 같은 것을 보아 멤버십 가입 시 성별을 반드시 기입해야 한다고 볼 수 있다.
# notmember.isna().sum()

비회원인데 멤버십 정보가 있는 데이터수:  308


,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6
5,블루,무료,NaN,NaN,NaN,NaN
11745,골드,무료,노블,NaN,NaN,NaN
18821,블루,골드,무료,노블,NaN,NaN
19683,무료,노블,NaN,NaN,NaN,NaN
20285,무료,NaN,NaN,NaN,NaN,NaN
49502,골드,무료,NaN,NaN,NaN,NaN
107693,블루,골드,무료,그린,NaN,NaN
118193,블루,골드,무료,NaN,NaN,NaN
124729,무료,싹틔우미,NaN,NaN,NaN,NaN
140112,블루,무료,그린,NaN,NaN,NaN


In [8]:
data = data.drop(notmember.loc[notmember['gender'].notna()].index).reset_index(drop=True)
data.shape

(1009243, 23)

### <font color="firebrick">변수 정리
기존 변수를 정의를 고려하여 새롭게 정리한다.
- member_yn 정의는 230826_DataCleansing에서 진행했음으로 언급하지 않는다.
- membership_type 구조는 아래와 같다.
  - 예술의 전당을 통해 예매하기 위해선 무료회원에 가입해야 하며 <br>
    무료회원 3개는 싹틔우미([무료, 싹틔우미(11)])와 일반([무료(12)]), 노블([무료, 노블(8)])로 표현되어 있다.
  - 유료회원 혜택이 중복(누적) 적용되기에 무료회원에 그린, 블루, 골드회원까지 가입하는데 제한이 없다. 최대 membership_type5열로 회원제를 표현한다.
  - 유료회원 중 법인회원은 discount_type이 `골드회원(법인A)`로 표현되어 있다.
  - [참고] https://www.sac.or.kr/site/main/membership/member_step

In [9]:
# 중복으로 가입해도 무료, 싹틔우미/노블, 그린, 블루, 골드로 membership_type_6는 불필요한 column이다.
data.drop('membership_type_6', axis=1, inplace=True)

In [10]:
# 무료회원 가입은 필수로 이루어지고 유료회원 가입은 선택사항이다.
member = data.query('member_yn=="Y"').iloc[:,2:7]
member.drop_duplicates().sort_values(by=member.columns.tolist()).reset_index(drop=True)

,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5
0,골드,무료,그린,NaN,NaN
1,골드,무료,노블,그린,NaN
2,골드,무료,노블,NaN,NaN
3,골드,무료,싹틔우미,그린,NaN
4,골드,무료,싹틔우미,NaN,NaN
5,골드,무료,NaN,NaN,NaN
6,무료,그린,NaN,NaN,NaN
7,무료,노블,그린,NaN,NaN
8,무료,노블,NaN,NaN,NaN
9,무료,싹틔우미,그린,NaN,NaN


In [11]:
member['free_membership'] = member.apply(lambda x: '싹틔우미' if '싹틔우미' in x.values else '노블' if '노블' in x.values else '일반', axis=1)
member['green'] = member.apply(lambda x: 1 if '그린' in x.values else 0, axis=1)
member['blue'] = member.apply(lambda x: 1 if '블루' in x.values else 0, axis=1)
member['gold'] = member.apply(lambda x: 1 if '골드' in x.values else 0, axis=1)

In [12]:
member['corporate'] = data.query('member_yn=="Y"')['discount_type'].apply(lambda x: 1 if "법인" in x else 0)
# member.query('corporate==1').iloc[:,:5].drop_duplicates()

In [13]:
# 전체 데이터를 병합한다.
data.drop(['membership_type_1', 'membership_type_2','membership_type_3', 'membership_type_4', 'membership_type_5'], 
          axis=1, inplace=True)
data = pd.concat([data, member.iloc[:,-5:]], axis=1)
data.head()

,age,gender,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,...,open_date,genre,running_time,intermission,member_yn,free_membership,green,blue,gold,corporate
0,50.0,F,2022-01-14,1512,2022-02-04,2000,3층 BOX9 10,10000,2,일반,...,2022-01-15,교향곡,120,15,Y,일반,1.0,0.0,1.0,0.0
1,50.0,M,2022-02-06,1615,2022-03-02,1930,1층 B블록12열 7,180000,0,일반,...,2022-01-10,독주,90,0,Y,일반,0.0,0.0,0.0,0.0
2,30.0,F,2018-11-24,1145,2019-03-23,2000,1층 A블록2열 1,144000,2,블루회원 할인20%,...,2018-11-19,교향곡,100,15,Y,일반,0.0,1.0,0.0,0.0
3,NaN,NaN,2019-10-20,1459,2019-11-09,1700,3층 E블록4열 8,0,0,초대권,...,2019-10-23,교향곡,100,15,N,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,2019-06-13,954,2019-07-23,2000,2층 D블록8열 4,0,0,초대권,...,2019-06-02,교향곡,120,20,N,NaN,NaN,NaN,NaN,NaN


In [14]:
# 선예매를 하지 않은 데이터, 비회원 데이터를 의미하게 결측치를 남겼다.
data.isna().sum()

age                 551469
gender              551469
tran_date                0
tran_time                0
play_date                0
play_st_time             0
seat                     0
price                    0
ticket_cancel            0
discount_type            0
performance_code         0
pre_open_date       366172
open_date                0
genre                    0
running_time             0
intermission             0
member_yn                0
free_membership     551469
green               551469
blue                551469
gold                551469
corporate           551469
dtype: int64

- performance_code, play_date, play_st_time이 동일해야 같은 공연을 의미한다. 쉽게 알아볼 수 있게 performance_label을 생성한다.

In [15]:
data = data.merge(data[['performance_code', 'play_date', 'play_st_time']]\
                  .drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'performance_label'}),
                  on=['performance_code', 'play_date', 'play_st_time'])
data.drop('performance_code', axis=1, inplace=True)

- ticket_cancel은 예매율/취소율을 구하기 위해 label을 0과 1로 변경한다.

In [16]:
data['ticket_cancel'] = data['ticket_cancel'].map({0:0,2:1})

- discount_type은 결측치가 없다.
  - 할인을 받지 않고 정가에 구매한 데이터는 '일반'으로 표기되어 있다.
  - label에 나와있는 할인율은 별도로 저장해두었다가 원가 계산, 공연 등급제 등을 유추하도록 한다.

In [17]:
data.discount_type.value_counts()

초대권                   347637
일반                    160309
기획사판매                 126387
골드회원 할인10%             51844
그린회원 할인5%              27008
                       ...  
싹틔우미 할인                    3
대학생 할인(본인만)30%             2
예술인패스카드 소지자(본인)20%         2
골드회원 할인30%_                2
블루회원 할인20%_                2
Name: discount_type, Length: 673, dtype: int64

In [18]:
# discount_type이 "일반"인 데이터를 살펴보면 천원 단위가 맞춰져 있으며 정가 구매로 보여진다.
# data.query('discount_type == "일반"').groupby(['performance_code','seat'])['price'].unique()
print('discount_type이 "일반"이나 천원 단위가 아닌 데이터수:', \
      data.query('discount_type=="일반"')['price'].astype(str).apply(lambda x: 1 if '000' not in x else 0).sum())

discount_type이 "일반"이나 천원 단위가 아닌 데이터수: 0


In [19]:
# 할인율과 정가구매, 초대권 사용 정보를 추출한다.
data['discount_rate'] = data[['price','discount_type']].apply(lambda x: 1 if x[0] == 0 and x[1] == "초대권" else
                                                                         0 if x[0] != 0  and x[1] == "일반" else
                                                                         x[1][x[1].index('%')-2:x[1].index('%')] if x[0] != 0 and '%' in x[1] else np.nan, axis=1)
data['discount_rate'] = data['discount_rate'].apply(lambda x: {')5':'5','인5':'5'}.get(x, x)).astype(float)

In [20]:
# 초대권 : 347636개
# 정가구매(일반) : 160309개
# 할인적용 : 314330개
# 초대권 외의 구매가격이 0인 데이터: 174020개
# 할인율을 알 수 없는 데이터: 12948개

data.discount_rate.value_counts(dropna=False)

1.0     347636
NaN     186968
0.0     160309
10.0     81845
5.0      49632
30.0     45977
20.0     45913
50.0     45355
40.0     28163
15.0     13734
25.0      2279
12.0      1104
60.0       234
75.0        39
35.0        28
80.0        27
Name: discount_rate, dtype: int64

- 위에서 구한 할인율을 바탕으로 원가를 복원할 수 있는 데이터는 복원한다.

In [21]:
data['origin_price'] = (data['price'] / (1 - data.discount_rate.apply(lambda x: x/100))).apply(lambda x: round(x, 2))

- seat을 그룹핑하는 것이 분석과제 중 하나로 column을 상세히 쪼갠다.
  - 이후 동일 좌석임을 알아보기 위해 seat_label을 생성한다.

In [22]:
# 블록이 BOX라 열이 없는 데이터는 열은 nan 처리해둔다.
data['층'] = data['seat'].apply(lambda x: x[:x.index('층')] if '층' in x else '합창석')
data['블록'] = data['seat'].str.split(expand=True)[1].apply(lambda x: x[:x.index('블록')] if '블록' in x else 
                                                                       x if 'BOX' in x else np.nan).str.strip()
data['열'] = data['seat'].apply(lambda x: x[x.index('블록')+2:x.index('열')] if '블록' in x else 
                                           np.nan if 'BOX' in x else x.rsplit()[-2][:-1]).str.strip()
data['좌석번호'] = data['seat'].apply(lambda x: x.rsplit()[-1]).str.strip()
# data[['seat','층','블록','열','좌석번호']]

In [23]:
# 동일 좌석임을 빠르게 확인하기 위해 seat_label을 만든다.
data = data.merge(data[['층','블록','열','좌석번호']].sort_values(by=['층','블록','열','좌석번호'])\
                  .drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'seat_label'}),
                  on=['층','블록','열','좌석번호'])
data.drop('seat', axis=1, inplace=True)

### <font color="firebrick">데이터 정제
정의와 맞지 않는 데이터를 제거하는 작업이다.
- age는 각 무료회원제 대상 연령과 부합해야 한다.
  - 싹틔우미는 7 ~ 24세, 일반은 25 ~ 68세, 노블은 69세이상으로 멤버십에 맞는 연령대를 갖지 않은 데이터는 오류이다.
  - 비회원이 무료회원의 할인을 받았거나 적합하지 않은 무료회원의 할인을 받은 데이터는 오류로 제거한다. 

In [24]:
data.drop(data.query('free_membership=="싹틔우미" and age >= 30').index, inplace=True)
print('데이터 수:', data.shape[0])
data.drop(data.query('free_membership=="일반" and (age == 10 or age > 60)').index, inplace=True)
print('데이터 수:', data.shape[0])
data.drop(data.query('free_membership=="노블" and age < 60').index, inplace=True)
print('데이터 수:', data.shape[0])

데이터 수: 991112
데이터 수: 980690
데이터 수: 980427


In [25]:
data.drop(data.loc[data.discount_type.apply(lambda x: '싹틔우미' in x)].query('free_membership != "싹틔우미"').index, inplace=True)
print('데이터수:', data.shape[0])
data.drop(data.loc[data.discount_type.apply(lambda x: '노블' in x)].query('free_membership != "노블"').index, inplace=True)
print('데이터수:', data.shape[0])

데이터수: 980384
데이터수: 980366


- discount_type에서 유료회원 혜택은 중복적용될 수 있되 다른 유료회원 혜택이 적용될 수 없음을 고려해 적합한 혜택이 적용된 데이터만을 남긴다.

In [26]:
# 유료회원별 적합한 혜택이 적용된 데이터만을 남긴다.
data.drop(data.loc[data.discount_type.apply(lambda x: '그린' in x)].query('green != 1').index, inplace=True)
print('데이터수:', data.shape[0])
data.drop(data.loc[data.discount_type.apply(lambda x: '블루' in x)].query('blue != 1').index, inplace=True)
print('데이터수:', data.shape[0])
data.drop(data.loc[data.discount_type.apply(lambda x: '골드' in x)].query('gold != 1 and corporate != 1').index, inplace=True)
print('데이터수:', data.shape[0])

데이터수: 980308
데이터수: 980235
데이터수: 980126


- pre_open_date이 예매시작일보다 빠른 데이터는 오류로 제거한다.

In [27]:
# 선예매일이 예매시작일보다 빠른 데이터를 제거한다. 
data.drop(data.query('pre_open_date > open_date').index, inplace=True)
print('데이터수:', data.shape[0])

데이터수: 977242


- trun_date가 선예매일이나 예매일보다 빠른 데이터는 오류이다.

In [28]:
data.drop(data.loc[data['pre_open_date'].notna()].query('pre_open_date > tran_date').index, inplace=True)
print('데이터수:', data.shape[0])

데이터수: 959068


In [29]:
data.drop(data.loc[data['pre_open_date'].isna()].query('open_date > tran_date').index, inplace=True)
print('데이터수:', data.shape[0])

데이터수: 929040


### <font color="firebrick">변수 생성
분석에 사용할 추가적인 변수를 생성한다.

- play_date, play_st_time으로부터 일자 정보를 생성한다.

In [30]:
data['play_year'] = data['play_date'].dt.year
data['play_month'] = data['play_date'].dt.month
data['play_weekday'] = data['play_date'].dt.weekday
data['play_time'] = data['play_st_time']//100 + data['play_st_time']%100/60

- open_date로부터 일자 정보를 생성한다.

In [31]:
data['open_year'] = data['open_date'].dt.year
data['open_month'] = data['open_date'].dt.month

- pre_open_date와 open_date 차이를 구해 선예매여부 및 선예매기간을 구한다.

In [32]:
data['pre_open_gap'] = (data['open_date'] - data['pre_open_date']).dt.days.fillna(-1)

- tran_date와 open_date 차이를 구해 얼마나 빨리 구매했는지 확인한다.

In [33]:
data['tran_gap'] = (data['tran_date'] - data['open_date']).dt.days.fillna(-1)

- play_date와 open_date 차이를 구한다.

In [34]:
data['open_gap'] = (data['play_date'] - data['open_date']).dt.days.fillna(-1)

- play_date와 tran_date 차이를 구해 취소가능성이 높은지 확인한다.

In [35]:
data['play_gap'] = (data['play_date'] - data['tran_date']).dt.days.fillna(-1)

- 해당 공연에서 몇 번째로 구매된 좌석인지 구한다.

In [36]:
data['tran'] = pd.to_datetime(data['tran_date'].astype(str)+data['tran_time'].astype(str).str.zfill(4), format='%Y-%m-%d%H%M')

In [37]:
min_times = data.groupby(['performance_label'])['tran'].min().rename('min').reset_index()
min_times = data[['performance_label','tran']].reset_index().merge(min_times, on='performance_label').set_index('index')
data['res_time'] = (min_times['tran'] - min_times['min']).dt.total_seconds()
data.drop('tran', axis=1, inplace=True)

In [38]:
data['res_time_rank'] = data.groupby('performance_label')['res_time'].rank(method='min')

In [39]:
data['res_time_rank_scaled'] = data.groupby('performance_label')['res_time_rank']\
                               .transform(lambda x: MinMaxScaler().fit_transform(x.values.reshape(-1, 1)).ravel())

### Save data

In [40]:
data.to_parquet('../data/DataCleansing.pqt', index=False)